In [1]:
#### run combined df again if any code changes occur (symbol change) including additions 


In [12]:
#### combines 7 days delivery files
#### reads market cap from screener
#### reads averages

#### this reads non_fno_wl, yymmdd_volume_3x_wl, yymmdd_tview_wl_above_500, yymmdd_tview_wl_below_500, sme_csv
 

#### heading for reports
#### title for reports

In [2]:
# importing necessary modules 

import os
import pandas as pd
import csv
from pathlib import Path

base_folder = Path.cwd()
print(base_folder)

c:\Users\birap\OneDrive\Desktop\Tradeview\tradingView\Python code for chatGpt


In [3]:
###  datapath for lenovo desktop

master_file_dir = Path.joinpath(base_folder, "Master File")

master_file_path = Path.joinpath(base_folder, "Master File", "CompanyMaster.csv")
directory = Path.joinpath(Path(base_folder).parents[0], "Data Files")
combined_directory =  Path.joinpath(Path(base_folder).parents[0], "Delivery Price Combined Files")


input_path =  Path.joinpath(base_folder, "Input File")
output_path = Path.joinpath(base_folder, "Output File")

In [4]:
column_order = ['NSE_Code','BSE_Code','Date','NSE_Delivery','BSE_Delivery']


non_fno_csv = os.path.join(output_path, '0_non_fno.csv')
fno_csv = os.path.join(output_path, 'fno.csv')
sme_csv = os.path.join(output_path, 'sme_csv.csv')

avgs_file = os.path.join(input_path, 'NSE BSE Averages.xlsx')

In [89]:
#### reading nse and bse averages
file_path = avgs_file

# Replace 'Sheet1' with the name of the sheet you want to read
#sheet_name = 'Sheet1'
nse_averages = "NSE Averages"
bse_averages = "BSE Averages"

# Read the Excel file into a DataFrame
#df = pd.read_excel(file_path, sheet_name=sheet_name)

#nse_avg_df = pd.read_excel(file_path, sheet_name = nse_averages, dtype=column_width, engine='openpyxl')
nse_avg_df = pd.read_excel(file_path, sheet_name=nse_averages)
nse_avg_df['Name'] = nse_avg_df['Name'].str.replace('_T$',"",regex=True)
nse_avg_df['Name'] = nse_avg_df['Name'].str.replace('_Z$',"",regex=True)
nse_avg_df['Name'] = nse_avg_df['Name'].str.replace(' SM$',"",regex=True)
nse_avg_df['Name'] = nse_avg_df['Name'].str.replace(' ST$',"",regex=True)


nse_avg_df['Name'] = nse_avg_df['Name'].str.strip()


bse_avg_df = pd.read_excel(file_path, sheet_name = bse_averages)
bse_avg_df['Code'] = bse_avg_df['Code'].astype(str)


####################reading market cap file

mktcap_file = os.path.join(master_file_dir, 'MarketCap from Screener.csv')
mktcap_df = pd.read_csv(mktcap_file)
mktcap_df['BSE Code'] = mktcap_df['BSE Code'].fillna(0)
mktcap_df['BSE Code'] = mktcap_df['BSE Code'].astype(int)
mktcap_df['NSE Code'] = mktcap_df['NSE Code'].fillna("")

mktcap_df.rename(columns= { 'BSE Code' : 'BSE_Code', 
                          'NSE Code': 'NSE_Code'}, inplace =True)

mktcap_df.loc[mktcap_df['NSE_Code'] == "", "NSE_Code"] = mktcap_df['BSE_Code']


In [93]:

###### combine 7 days price files

no_days = 7

file_names = sorted([file for file in os.listdir(combined_directory) if file.lower().endswith(("CSV","csv"))
                        and file.startswith("combined")], reverse=True)[:no_days]

print(file_names)

count = 1

for file_name in file_names:
    file_path = os.path.join(combined_directory,file_name)
    csv_file_df = pd.read_csv(file_path)
    
    if count == 1:
        combined_df = csv_file_df
    else:
        combined_df = pd.concat([combined_df, csv_file_df], ignore_index=True)
    
    count += 1
    #print(count)

combined_df['BSE_Code'] = combined_df['BSE_Code'].fillna(0)
combined_df['BSE_Code'] = combined_df['BSE_Code'].astype(int)
combined_df['NSE_Code'] = combined_df['NSE_Code'].fillna("")
combined_df['Company Name'] = combined_df['Company Name'].fillna("")
combined_df['NSE_Code'] = combined_df.apply(replace_nse_code, axis=1)

# Assuming your DataFrame is named combined_df
report_date = combined_df['Date'].max()
report_date = str(report_date)

print(report_date)

['combined_231020.csv', 'combined_231010.csv', 'combined_230810.csv', 'combined_230420.csv', 'combined_190124.csv']
231020


In [91]:
def find_corresponding_codes(master_file_path, user_input):
    nse_code = ""
    bse_code = ""
    co_name = ""

    user_input = user_input.lower()  # Convert user input to lowercase for comparison

    with open(master_file_path, 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            if row[1].lower() == user_input:  # NSE code match (case-insensitive)
                nse_code = row[1]
                bse_code = str(row[2])
                co_name = row[0]
                break
            elif row[2].lower() == user_input:  # BSE code match (case-insensitive)
                nse_code = row[1]
                bse_code = str(row[2])
                co_name = row[0]
                break

    return nse_code, bse_code,co_name

def format_value(value):
    return '{:,.0f}'.format(value)

def format_decimal(x):
    return '{:,.2f}'.format(x)

def averages(scrip_code):
    ema10 = [0]  # Default value for ema10
    ema20 = [0]  # Default value for ema20
    ema50 = [0]  # Default value for ema50
    ema200 = [0]  # Default value for ema200
    
    
        
    if type(scrip_code) == int:
        scrip_code = str(scrip_code)
        averages_df = bse_avg_df[bse_avg_df['Code'] == scrip_code]
    else:
        scrip_code = scrip_code.upper()
        averages_df = nse_avg_df[nse_avg_df['Name'] == scrip_code]
    
    if not averages_df.empty:
        ema10 = averages_df['ema10'].values.tolist()
        ema20 = averages_df['ema20'].values.tolist()
        ema50 = averages_df['ema50'].values.tolist()
        ema200 = averages_df['ema200'].values.tolist()
    
    return ema10, ema20, ema50, ema200


def get_mktcap(scrip_code):
    
    if type(scrip_code) == int:
        #scrip_code = str(scrip_code)
        scrip_mktcap_df = mktcap_df[mktcap_df['BSE_Code'] == scrip_code]
    else:
        scrip_code = scrip_code.upper()
        scrip_mktcap_df = mktcap_df[mktcap_df['NSE_Code'] == scrip_code]
        
    #scrip_mktcap_df.info()
    
    if not scrip_mktcap_df.empty:
        scrip_mktcap = scrip_mktcap_df['Market Capitalization'].values.tolist()
        
    else:
        scrip_mktcap = [0]
    
    return scrip_mktcap


def get_networth(scrip_code):
    
    if type(scrip_code) == int:
        #scrip_code = str(scrip_code)
        scrip_mktcap_df = mktcap_df[mktcap_df['BSE_Code'] == scrip_code]
    else:
        scrip_code = scrip_code.upper()
        scrip_mktcap_df = mktcap_df[mktcap_df['NSE_Code'] == scrip_code]
        
    #scrip_mktcap_df.info()
    
    if not scrip_mktcap_df.empty:
        scrip_networth = scrip_mktcap_df['Net worth'].values.tolist()
        
    else:
        scrip_networth = [0]
    
    return scrip_networth
     
    
    
def replace_nse_code(row):
    if pd.isna(row['NSE_Code']) or row['NSE_Code'] == "":
        return row['BSE_Code']
    else:
        return row['NSE_Code']

# Apply the custom function to the NSE_Code column

def list_process(list_name):
    list_name_lower = [item.lower() for item in list_name]
    nse_code_df = combined_df[combined_df['NSE_Code'].str.lower().isin(list_name_lower)]
    
    numeric_list_name = []
    
    for value in list_name:
        if value.isdigit():
            numeric_list_name.append(int(value))
    
    bse_code_df = combined_df[combined_df['BSE_Code'].isin(numeric_list_name)]
    final_code_df = pd.concat([nse_code_df,bse_code_df], ignore_index=True)
    final_code_df['NSE_Code'] = final_code_df.apply(replace_nse_code, axis=1)
    
    final_code_df.sort_values(by=['NSE_Code', 'Date'], ascending=[True, False], inplace=True)
    grouped_df = final_code_df.groupby('NSE_Code')
    
    for name, group in grouped_df:
        
        ema10, ema20, ema50, ema200 = averages(str(name))
        ema_str = "ema10 / ema20 / ema50 / ema200"
    
        print("")
        print(f"Company Name: {group['Company Name'].iloc[0]} (NSE Code: {name}, BSE Code: {group['BSE_Code'].iloc[0]})")
        print(f"{ema_str}     {ema10[0]}  {ema20[0]}  {ema50[0]}  {ema200[0]}")
        print("")

        # Format the columns before printing
        group['NSE_Delivery'] = group['NSE_Delivery'].apply(lambda x: '{:,.0f}'.format(x))
        group['BSE_Delivery'] = group['BSE_Delivery'].apply(lambda x: '{:,.0f}'.format(x))
        group['Total_Delivery'] = group['Total_Delivery'].apply(lambda x: '{:,.0f}'.format(x))

        # Print the header with column names excluding 'NSE_Code' and 'BSE_Code'
        header = ['Date', 'NSE_Delivery', 'BSE_Delivery', 'Total_Delivery', 'Price']
    
        # Print the table with manual formatting
        print(f"{'Date':<8}{'NSE_Delivery':>15}{'BSE_Delivery':>15}{'Total_Delivery':>20}{'Price':>15}")
        for _, row in group[header].iterrows():
            print(f"{row['Date']:<8}{row['NSE_Delivery']:>15}{row['BSE_Delivery']:>15}{row['Total_Delivery']:>20}{row['Price']:>15.2f}")

        # Add two blank spaces between groups
        print("\n")
        

    
    return final_code_df
    
    
def list_process_new(list_name, header):
    
    print(header)
    
    for company in list_name:
    # Filter the DataFrame for the current company
    
        if company.isdigit():
            company_int = int(company)
            company_data = combined_df[combined_df['BSE_Code'] == company_int]
            #print(company)
        else:
            company = company.upper()
            company_data = combined_df[combined_df['NSE_Code'] == company]
            #print(company)

        # Check if the DataFrame is not empty
        if not company_data.empty:
            # Extract relevant information
            company_name = company_data['Company Name'].values[0]
            nse_code = company_data['NSE_Code'].values[0]
            bse_code = company_data['BSE_Code'].values[0]

            ema10,ema20,ema50,ema200 = averages(nse_code)
            ema_str = "ema10 / ema20 / ema50 / ema200"
            
            mktcap_value = get_mktcap(nse_code)
            networth_value = get_networth(nse_code)
            
            print(f"Company Name: {company_name} (NSE Code: {nse_code}, BSE Code: {bse_code}, Mktcap:{mktcap_value[0]:,.0f}, Networth:{networth_value[0]:,.0f})")

            print("")
        
            print(f"{ema_str}     {ema10[0]}  {ema20[0]}  {ema50[0]}  {ema200[0]}")
            print("")

            # Format and print the specified columns with additional spaces
            formatted_columns = ['Date', 'NSE_Delivery', 'BSE_Delivery', 'Total_Delivery', 'Price']
            formatted_data = company_data[formatted_columns].copy()
            formatted_data['NSE_Delivery'] = formatted_data['NSE_Delivery'].apply(lambda x: f"{int(x):,}")
            formatted_data['BSE_Delivery'] = formatted_data['BSE_Delivery'].apply(lambda x: f"{int(x):,}")
            formatted_data['Total_Delivery'] = formatted_data['Total_Delivery'].apply(lambda x: f"{int(x):,}")
            formatted_data['Price'] = formatted_data['Price'].apply(lambda x: f"{x:.2f}")

            # Adjust the spacing between columns
            formatted_data_str = formatted_data.to_string(index=False, 
                                                      col_space={'Date':9,
                                                                 'NSE_Delivery':15,
                                                                 'BSE_Delivery':15,
                                                                 'Total_Delivery': 15, 
                                                                 'Price': 10
                                                                 }
                                                        )

            print(formatted_data_str)
            # Add a line break between companies
            print("\n" + "=" * 50 + "\n")
            print("")
        else:
            print(f"No data found for {company}\n")

    
def add_nse_prefix(df, column_name):
    # Create a copy of the DataFrame to avoid SettingWithCopyWarning
    df = df.copy()

    # Add "NSE:" prefix to the specified column
    df.loc[:, column_name] = "NSE:" + df[column_name].astype(str)

    return df


In [201]:
def print_formatted_rows(stock_list, title):
    # Print the header
    header = "Company Name".ljust(35) + "StockCode".ljust(10) + "Close".rjust(10) + "PrevClose".rjust(10) + \
             "MktCap".rjust(10) + "EMA10".rjust(10) + "EMA20".rjust(10) + "EMA50".rjust(10) + "EMA200".rjust(10)
    print(title)
    print("")
    print("")
    print(header)
    print("")
    print("")

    line_counter = 0
    
    for stock in stock_list:
        if stock.isdigit():
            stock_code = int(stock)
            df_row = combined_df[combined_df['BSE_Code'] == stock_code]
        else:
            stock_code = stock.upper()
            df_row = combined_df[combined_df['NSE_Code'] == stock_code]

        if not df_row.empty:
            company_name = df_row['Company Name'].values[0]
            stock_code = df_row['NSE_Code'].values[0]
            price = df_row['Price'].values[0]
            prevclose = df_row['PrevClose'].values[0]

            ema10, ema20, ema50, ema200 = averages(stock_code)
            ema10 = ema10[0]
            ema20 = ema20[0]
            ema50 = ema50[0]
            ema200 = ema200[0]

            mktcap_value = get_mktcap(stock_code)
            mktcap = mktcap_value[0]

            # Format the values with ljust and rjust
            formatted_row = company_name.ljust(35) + str(stock_code).ljust(10) + \
                             f"{price:.2f}".rjust(10) + f"{prevclose:.2f}".rjust(10) + \
                             f"{mktcap:,.0f}".rjust(10) + f"{ema10:.2f}".rjust(10) + \
                             f"{ema20:.2f}".rjust(10) + f"{ema50:.2f}".rjust(10) + f"{ema200:.2f}".rjust(10)

            print(formatted_row)
            
            line_counter += 1

            # Insert a blank line after every 5 lines
            if line_counter % 5 == 0:
                print()

In [94]:
### reading csv of top 50 by value and volume in delivery

non_fno_df_in = pd.read_csv(non_fno_csv)
fno_df = pd.read_csv(fno_csv)
sme_df_in = pd.read_csv(sme_csv)

"""It fills the missing (NaN) values in the 'NSE_Code' column of the DataFrame non_fno_df with values 
from the 'BSE_Code' column converted to strings.

The .astype(str) part is used to convert the values in the 'BSE_Code' column to strings. 
This is necessary because the 'NSE_Code' column likely contains strings (object dtype), and 
you need to ensure that the replacement values are of the same data type.

The fillna method is used to perform the filling operation. By setting the inplace parameter to True, 
the operation is performed directly on the DataFrame without the need to assign the result back to the DataFrame."""


def fill_missing_nse_code(data_frame):
    # Convert non-finite 'BSE_Code' values to integers while replacing NaN with a placeholder
    placeholder_int = -1  # Placeholder integer value for NaN
    data_frame['BSE_Code'] = data_frame['BSE_Code'].fillna(placeholder_int).astype(int)
    
    # Convert 'BSE_Code' values to strings and fill missing 'NSE_Code' values
    data_frame['NSE_Code'].fillna(data_frame['BSE_Code'].astype(str), inplace=True)

    return data_frame


non_fno_df = fill_missing_nse_code(non_fno_df_in)
sme_df = fill_missing_nse_code(sme_df_in)

#non_fno_df['NSE_Code'].fillna(non_fno_df['BSE_Code'].astype(int).astype(str), inplace=True)
#sme_df['NSE_Code'].fillna(sme_df['BSE_Code'].astype(int).astype(str), inplace=True)

# Get the unique 'NSE_Code' values as a list
unique_non_fno = non_fno_df['NSE_Code'].unique().tolist()
sme_list = sme_df['NSE_Code'].unique().tolist()

focus_list = [ '531112', '538734', '543953', '543971', 'adanipower', 'amber', 'anantraj', 'angelone', 'anup', 'apollo', 
           'arvind', 'arvindfasn', 'bajajhind', 'bclind', 'bluejet', 'bluestarco', 'brigade', 'bse', 'cdsl', 'chembond', 
           'cignititec', 'creditacc', 'cupid', 'daawat', 'dbrealty', 'dharmaj', 'diamondyd', 'elecon', 'electcast', 
           'emil', 'focus', 'gabriel', 'gensol', 'genuspower', 'hblpower', 'hirect', 'hitech', 'hpl', 'idea', 'hcc',
           'ifglexpor', 'iiflsec', 'ikio', 'indotech', 'inoxwind', 'jindalsaw', 'jkcement', 'jklakshmi', '530129',
           'jktyre', 'jpassociat', 'jppower', 'jwl', 'kapston', 'kesoramind', 'kpittech', 'l&tfh', 'lalpathlab', 'tv18brdcst',
           'lodha', 'marine', 'mblinfra', 'mcx', 'newgen', 'paytm', 'peninland', 'pfs', 'repcohome','ndrauto','magson',
           'phoenixltd', 'pplpharma', 'premexpln', 'prestige', 'pyramid', 'radico', 'rategain', 'recltd', '522101',
           'rkforge', 'rpglife', 'sadhnaniq', 'sanghvimov', 'satin', 'sdbl', 'senco', 'sepc', 'shriramfin','shrirampps', 
            'skygold', 'solarinds', 'sonatsoftw', 'spandana', 'suzlon', 'tatamotors', 'thomascook', 'shripiston',
           'titagarh', 'tril', 'unionbank', 'vijaya', 'walchannag', 'welspunind', 'zaggle', 'zeemedia', 'zomato',
           'hariompipe', 'semac', 'tci', '540358', 'icil', '20microns', 'prsmjohnsn', 'poonawalla', 'cello', 'hcc', 
          'bikaji', 'neulandlab', 'jppower', 'trent']

In [95]:
dely_3x = os.path.join(output_path, f'_{report_date}_0_volume_3x_wl.csv')
dely_3x_df = pd.read_csv(dely_3x)
dely_3x_df.head()

,Symbol
0,NSE:PLAZACABLE
1,NSE:541701
2,NSE:514010
3,NSE:512329
4,NSE:GKWLIMITED


In [96]:

#dely_3x_list = dely_3x_df.tolist()
# Extract the 'ScripCode' column from the DataFrame
dely_3x_list = dely_3x_df['Symbol'].tolist()

cleaned_list = [stock.replace('NSE:', '') for stock in dely_3x_list]
dely_3x_list = cleaned_list

# Extract the first 50 records of the 'ScripCode' column from the DataFrame
#dely_3x_list = dely_3x_df['ScripCode'].head(50).tolist()

#dely_3x_list

In [97]:
### above 500 cr

# percent_new_high = os.path.join(output_path, f'_{report_date}_0_tview_wl_above_500.csv')
percent_new_high =r"C:\Users\birap\OneDrive\Desktop\Tradeview\tradingView\Python code for chatGpt\Output File\_231221_0_tview_wl_above_500.csv"
percent_new_high_df = pd.read_csv(percent_new_high)

percent_new_high_df.head()

,Ticker
0,NSE:VERTOZ
1,NSE:ARMANFIN
2,NSE:TARSONS
3,NSE:LOKESHMACH
4,NSE:PHANTOMFX


In [98]:
# Replace "NSE:" with an empty string in the 'Ticker' column
percent_new_high_df['Ticker'] = percent_new_high_df['Ticker'].str.replace('NSE:', '')

# Assuming percent_new_high_df is your DataFrame

# Replace "NSE:" with an empty string in the 'Ticker' column
percent_new_high_df['Ticker'] = percent_new_high_df['Ticker'].str.replace('NSE:', '')

# Print the updated DataFrame
#print(percent_new_high_df)

# Convert the 'Ticker' column to a list
ticker_list = percent_new_high_df['Ticker'].tolist()

#print(ticker_list)


#### below 500 cr

# percent_below = os.path.join(output_path, f'_{report_date}_0_tview_wl_below_500.csv')

# percent_below_df = pd.read_csv(percent_below)
percent_below = r"C:\Users\birap\OneDrive\Desktop\Tradeview\tradingView\Python code for chatGpt\Output File\_231221_0_tview_wl_below_500.csv"
percent_below_df = pd.read_csv(percent_below)


In [99]:
percent_below_df.head()

,Ticker
0,NSE:SIKKO
1,NSE:STARTECK
2,NSE:HINDCON
3,NSE:CORALFINAC
4,NSE:VISESHINFO


In [100]:
percent_below_df['Ticker'] = percent_below_df['Ticker'].str.replace("NSE:","")
ticker_below = percent_below_df["Ticker"].tolist()

In [101]:
import openpyxl

file_path = "C:\\Users\\sarda\\Dropbox\\TS Personal\\Trade Files\\Consolidated Trade File from 1st April 2022.xlsx"
def read_sheet_pivots(file_path, sheet_name):
    data = []
    workbook = openpyxl.load_workbook(file_path, read_only=True)
    sheet = workbook[sheet_name]
    
    row_number = 28
    column_m = 'M'
    column_n = 'N'
    
    
    while True:
        cell_m = sheet[column_m + str(row_number)].value
        cell_n = sheet[column_n + str(row_number)].value
        
        if cell_m is None and cell_n is None:
            break
        
        # Convert the cell values to strings and store them in the list
        if cell_m is not None:
            m_value = str(cell_m)
            if m_value != '0' and (not m_value.isdigit() or m_value.startswith('5')):
                data.append(m_value)
        
        if cell_n is not None:
            n_value = str(cell_n)
            if n_value != '0' and (not n_value.isdigit() or n_value.startswith('5')):
                data.append(n_value)
        
        row_number += 1
    
    return data

# Example usage

sheet_name = 'Pivots'
holding_list = read_sheet_pivots(file_path, sheet_name)
print(holding_list)


##saving the holdings to csv for uploading to chartink and trading view

# Specify the name of the CSV file you want to create
csv_file_name = output_path + "0 My_Holdings.csv"

# Write the data to the CSV file
with open(csv_file_name, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Company"])  # Optional: Write a header row
    writer.writerows([[item] for item in holding_list])


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\sarda\\Dropbox\\TS Personal\\Trade Files\\Consolidated Trade File from 1st April 2022.xlsx'

In [102]:
# Remove duplicates and sort the list in-place
#my_list = list(set(my_list))
#my_list.sort()

list_below_1000 = ['522101','530129','538734','540358','540358','543953','543971','20microns','annapurna',
                   'arihantsup','chembond','dharmaj','global','indotech','kapston','kotharipet','magson',
                   'mblinfra','modisonltd','ndrauto','pyramid','semac','skygold','venusrem','walchannag']

list_below_1000 = list(set(list_below_1000))
list_below_1000.sort()


list_1k_10k = ['531112','542669','543620','anantraj','anup','apollo','arvind','arvindfasn','bajajhind','bancoindia',
               'bclind','bluejet','cignititec','cupid','dbrealty','diamondyd','dollar','electcast','emil','focus',
               'forcemot','gabriel','gensol','genuspower','hariompipe','hblpower','hcc','hirect','hitech','hpl',
               'icil','ifglexpor','iiflsec','ikio','inoxwind','jklakshmi','jktyre','jpassociat','jppower','kalamandir',
               'kesoramind','kfintech','kpigreen','ltfoods','lumaxtech','maninds','marine','moil','mvgjl','neulandlab',
               'newgen','paragmilk','peninland','pfs','pokarna','premexpln','prsmjohnsn','ramky','rategain','repcohome',
               'rpglife','sadhnaniq','sanghvimov','satin','sdbl','senco','sepc','shripiston','shrirampps','skipper',
               'somanycera','spandana','tci','tdpowersys','thomascook','tril','tv18brdcst','vijaya','zaggle',
               'zeemedia','awhcl','indramedco','marksans']

list_1k_10k = list(set(list_1k_10k))
list_1k_10k.sort()

list_above_10k = ['adanipower','amber','angelone','bikaji','bluestarco','brigade','bse','cdsl','cello','concordbio',
                  'creditacc','elecon','idea','jindalsaw','jkcement','jwl','kpittech','l&tfh','lalpathlab','lodha',
                  'mcx','patanjali','paytm','phoenixltd','piind','poonawalla','pplpharma','prestige','radico',
                  'recltd','rkforge','shriramfin','solarinds','sonatsoftw','suzlon','tatamotors','titagarh','trent',
                  'unionbank','welspunind','zomato','medanta','swanenergy','welcorp']

list_above_10k = list(set(list_above_10k))
list_above_10k.sort()


In [103]:
heading_non_fno = "          Top 50 7-day deliveries non_fno " + report_date
heading_sme = "          Top 50 7-day deliveries sme " + report_date

heading_3_percent = "          7-day deliveries up_3_percent or new_highs above 500 cr " + report_date
heading_3_percent_below = "          7-day deliveries up_3_percent or new_highs below 500 cr " + report_date

heading_3x = "          7-day deliveries 3x dely volume " + report_date
heading_holding = "          7-day deliveries for holdings " + report_date

heading_above_10k = "          7-day deliveries mktcap above 10k cr " + report_date
heading_below_1000 = "          7-day deliveries mktcap below 1000 cr " + report_date
heading_1k_10k = "            7-day deliveries mktcap 1k - 10k cr " + report_date

In [104]:
title_non_fno = "Non_fno top deliveries " + report_date + " with averages"
title_sme = "SME top deliveries " + report_date + " with averages"

title_3_percent = "         up_3_percent or new_highs above 500 cr " + report_date + " with averages"
title_3_percent_below = "         up_3_percent or new_highs below 500 cr " + report_date + " with averages"

title_3x = "          3x dely volume deliveries " + report_date + " with averages"
title_holding = "          7-day deliveries for holdings " + report_date + " with averages"

title_above_10k = "         Top deliveries mktcap above 10k cr " + report_date + " with averages"
title_below_1000 = "         Top deliveries mktcap below 1000 cr " + report_date + " with averages"
title_1k_10k = "          Top deliveries mktcap 1k - 10k cr " + report_date + " with averages"

In [105]:
ticker_list

['VERTOZ',
 'ARMANFIN',
 'TARSONS',
 'LOKESHMACH',
 'PHANTOMFX',
 'HINDCOPPER',
 'IMPAL',
 'INDOBORAX',
 'VINYAS',
 'NAVKARCORP',
 'SHYAMMETL',
 'SANGHVIMOV',
 'SANGAMIND',
 'ZENSARTECH',
 'APCL',
 'FCSSOFT',
 'BEML',
 'PCJEWELLER',
 'EMKAYTOOLS',
 'SALASAR',
 'MANALIPETC',
 'ZIMLAB',
 'GOLDIAM',
 'MOREPENLAB',
 'VIKASECO',
 'INOXWIND',
 'MSTCLTD',
 'VERANDA',
 'NECLIFE',
 'MTNL',
 'THYROCARE',
 'DEEPENR',
 'METROPOLIS',
 'GAEL',
 'HERITGFOOD',
 'SJVN',
 'COFFEEDAY',
 'HBLPOWER',
 'IIFLSEC',
 'INOXGREEN',
 'IRFC',
 'JASH',
 'GUFICBIO',
 'SATIN',
 'IRB',
 'TFCILTD',
 'ORIANA',
 'IRCTC',
 'WOCKPHARMA',
 'AGSTRA',
 'CHEMPLASTS',
 'ARIHANTSUP',
 'SNOWMAN',
 'WINDMACHIN',
 'GALLANTT',
 'TBZ',
 'HMVL',
 'SONACOMS',
 'DELHIVERY',
 'COCHINSHIP',
 'EXCELINDUS',
 'GEOJITFSL',
 'RPOWER',
 'RATEGAIN',
 'TTKHLTCARE',
 'NLCINDIA',
 'ZUARIIND',
 'RPSGVENT',
 'MARKSANS',
 'MASTEK',
 'SYNCOMF',
 'RAMCOSYS',
 'VENUSREM',
 'NATIONALUM',
 'ACE',
 'AUROPHARMA',
 'GLOBAL',
 'KELLTONTEC',
 'PRICOLLTD',
 'THE

In [237]:
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, PageBreak

def create_table_with_header(df, add_column_names=False, table_width=None):
    """create a table with header and data"""
    table_data = [df.columns.tolist()] + df.values.tolist()
        
    style = [
        ('ALIGN', (0, 0), (0, -1), 'LEFT'),  # Left align first column
        ('ALIGN', (1, 0), (-1, -1), 'RIGHT'),  # Right align remaining columns
        ('FONTSIZE', (1, 1), (-1, -1), 9),
        ('LEADING', (0, 0), (-1, -1), 8),  # Set leading for all cells to a smaller value
    ]

    t = Table(table_data, repeatRows=1, colWidths=[100, 80, 80, 80, 60])
    column_header_style = [('BACKGROUND', (0, 0), (-1, 0), colors.lightgrey),
                        ('TEXTCOLOR', (0, 0), (-1, 0), colors.black),
                        ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
                        ('FONTSIZE', (1, 1), (-1, -1), 9),
                        ]
    
    t.setStyle(column_header_style)
    t.setStyle(style)
    
    # Remove table borders
    t.setStyle([('LINEBELOW', (0, 0), (-1, 0), 0, colors.white),  # Hide line below header
                ('GRID', (0, 0), (-1, -1), 0, colors.white)])      # Hide grid lines
    
    return t

def add_headers_and_footers(canvas, doc):
    """Add header and footer here"""
    page_num = canvas.getPageNumber()
    text = f"Page {page_num}"
    footer_style = getSampleStyleSheet()['Normal']
    footer_style.alignment = 1  # 0=Left, 1=Center, 2=Right
    
    footer = Paragraph(text, footer_style)
    w, h = footer.wrap(doc.width, doc.bottomMargin - 10)  # Adjusted bottom margin
    footer.drawOn(canvas, doc.leftMargin, doc.bottomMargin - h)

    # Change title text color to white for pages after the first one
    if page_num > 1:
        title_style = getSampleStyleSheet()['Title']
        title_style.textColor = colors.white

def list_process_to_pdf(list_name, header, output_path):
    left_margin = 20
    right_margin = 20
    top_margin = 10
    bottom_margin = 20  # Add a bottom margin
    page_width = A4[0]  # Get width from letter page size
    page_height = A4[1] - top_margin - bottom_margin # Adjust height for margins

    # Create a SimpleDocTemplate with adjusted page size and margins
    doc = SimpleDocTemplate(
        str(output_path),
        pagesize=(page_width, page_height),
        leftMargin=left_margin,
        rightMargin=right_margin,
        topMargin=top_margin,
        bottomMargin=bottom_margin
    )

    company_no = 1
    # Create a list to hold all elements
    elements = []
    
    # header
    styles = getSampleStyleSheet()
    h2_style = styles['Title']
    h2_style.fontSize = 10
    h2_style.Name = 'Arial'
    h2_style.leading = 9
    main_title = Paragraph(header, h2_style)
    elements.extend([main_title])
    for company in list_name:
        # to adjust only 5 company in a single page
        if company_no > 5:
            company_no = 1
            elements.extend([PageBreak()])
        company_no +=1
        if company.isdigit():
            company_int = int(company)
            company_data = combined_df[combined_df['BSE_Code'] == company_int]
        else:
            company = company.upper()
            company_data = combined_df[combined_df['NSE_Code'] == company]
        if not company_data.empty:
            company_name = company_data['Company Name'].values[0]
            nse_code = company_data['NSE_Code'].values[0]
            bse_code = company_data['BSE_Code'].values[0]

            ema10, ema20, ema50, ema200 = averages(nse_code)
            ema_str = "ema10 / ema20 / ema50 / ema200"

            mktcap_value = get_mktcap(nse_code)
            styles = getSampleStyleSheet()

            # adding company info to pdf
            txt = f"Company Name: {company_name} (NSE Code: {nse_code}, BSE Code: {bse_code}, Mktcap: {mktcap_value[0]:,.0f})"
            h2_style = styles['Normal']
            h2_style.fontSize = 9
            h2_style.bold = True
            h2_style.alignment = 0
            main_title = Paragraph(txt, h2_style)
            elements.extend([main_title])

            # ema values
            txt =f"{ema_str} == {ema10[0]}    {ema20[0]}    {ema50[0]}    {ema200[0]}"
            main_title = Paragraph(txt, h2_style)
            elements.extend([main_title])
            # adding space
            elements.extend([Spacer(1, 24)])
            formatted_columns = ['Date', 'NSE_Delivery', 'BSE_Delivery', 'Total_Delivery', 'Price']
            formatted_data = company_data[formatted_columns].copy()
            formatted_data['NSE_Delivery'] = formatted_data['NSE_Delivery'].apply(lambda x: f"{int(x):,}")
            formatted_data['BSE_Delivery'] = formatted_data['BSE_Delivery'].apply(lambda x: f"{int(x):,}")
            formatted_data['Total_Delivery'] = formatted_data['Total_Delivery'].apply(lambda x: f"{int(x):,}")
            formatted_data['Price'] = formatted_data['Price'].apply(lambda x: f"{x:.2f}")

            formatted_data_str = formatted_data.to_string(index=False, col_space={'Date': 9, 'NSE_Delivery': 15, 'BSE_Delivery': 15,
                                                                                   'Total_Delivery': 15, 'Price': 10})
            table_width = doc.width - doc.leftMargin - doc.rightMargin
            table = create_table_with_header(formatted_data, add_column_names=True, table_width=table_width)
            elements.append(table)
            # adding space
            elements.extend([Spacer(0.5, 24)])
        else:
            txt=f"No data found for {company}\n"
            main_title = Paragraph(txt, h2_style)
            elements.extend([main_title])
        
    doc.build(elements, onFirstPage=add_headers_and_footers, onLaterPages=add_headers_and_footers)

In [278]:
def format_price(value):
    return f"{value:.2f}" if not pd.isna(value) else ""

def format_prevclose(value):
    return f"{value:.2f}" if not pd.isna(value) else ""

def format_mktcap(value):
    return f"{value:,.0f}" if not pd.isna(value) else ""

def format_ema(value):
    return f"{value:.2f}" if not pd.isna(value) else ""

def get_formatted_dataframe(stock_list):
    results_df = pd.DataFrame(columns=['Company Name', 'NSE Code', 'Price', 'PrevClose', 'MktCap', 'EMA10', 'EMA20', 'EMA50', 'EMA200'])

    for stock in stock_list:
        if stock.isdigit():
            stock_code = int(stock)
            df_row = combined_df[combined_df['BSE_Code'] == stock_code]
        else:
            stock_code = stock.upper()
            df_row = combined_df[combined_df['NSE_Code'] == stock_code]

        if not df_row.empty:
            company_name = df_row['Company Name'].values[0]
            stock_code = df_row['NSE_Code'].values[0]
            price = df_row['Price'].values[0]
            prevclose = df_row['PrevClose'].values[0]

            ema10, ema20, ema50, ema200 = averages(stock_code)
            ema10 = ema10[0]
            ema20 = ema20[0]
            ema50 = ema50[0]
            ema200 = ema200[0]

            mktcap_value = get_mktcap(stock_code)
            mktcap = mktcap_value[0]

            # Format the values with ljust and rjust
            row_df = pd.DataFrame({
                'Company Name': company_name,
                'NSE Code': stock_code,
                'Price': price,
                'PrevClose': prevclose,
                'MktCap': mktcap,
                'EMA10': ema10,
                'EMA20': ema20,
                'EMA50': ema50,
                'EMA200': ema200
            }, index=[0])
            
            row_df['Company Name'] = row_df['Company Name'].str.ljust(35)
            row_df['NSE Code'] = row_df['NSE Code'].astype(str).str.ljust(10)
            row_df['Price'] = row_df['Price'].map('{:.2f}'.format).str.rjust(10)
            row_df['PrevClose'] = row_df['PrevClose'].map('{:.2f}'.format).str.rjust(10)
            row_df['MktCap'] = row_df['MktCap'].map('{:,.0f}'.format).str.rjust(10)
            row_df['EMA10'] = row_df['EMA10'].map('{:.2f}'.format).str.rjust(10)
            row_df['EMA20'] = row_df['EMA20'].map('{:.2f}'.format).str.rjust(10)
            row_df['EMA50'] = row_df['EMA50'].map('{:.2f}'.format).str.rjust(10)
            row_df['EMA200'] = row_df['EMA200'].map('{:.2f}'.format).str.rjust(10)

            # Concatenate the row_df to results_df
            results_df = pd.concat([results_df, row_df], ignore_index=True)

    return results_df

def create_table_with_header_formatted(df, add_column_names=False, table_width=None):
    """create a table with header and data"""
    table_data = [df.columns.tolist()] + df.values.tolist()

    for i in range(6, len(table_data) + 2, 6):  # Include the last possible index
        table_data.insert(i, [''] * len(df.columns))

    style = [
        ('ALIGN', (0, 0), (1, -1), 'LEFT'),  # Left align first two columns
        ('ALIGN', (2, 0), (-1, -1), 'RIGHT'),  # Right align remaining columns
        ('FONTSIZE', (1, 1), (-1, -1), 9),
        ('LEADING', (0, 0), (-1, -1), 8),  # Set leading for all cells to a smaller value
    ]

    t = Table(table_data, repeatRows=1)
    column_header_style = [('BACKGROUND', (0, 0), (-1, 0), colors.lightgrey),
                        ('TEXTCOLOR', (0, 0), (-1, 0), colors.black),
                        ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
                        ('FONTSIZE', (1, 1), (-1, -1), 9),
                        ]
    
    t.setStyle(column_header_style)
    t.setStyle(style)
    
    # Remove table borders
    t.setStyle([('LINEBELOW', (0, 0), (-1, 0), 0, colors.white),  # Hide line below header
                ('GRID', (0, 0), (-1, -1), 0, colors.white)])      # Hide grid lines
    
    return t

def print_formatted_rows_pdf(stock_list, title, output_path):
    # Print the header
    left_margin = 20
    right_margin = 20
    top_margin = 10
    bottom_margin = 20  # Add a bottom margin
    page_width = A4[0] + 35 # Get width from letter page size
    page_height = A4[1] - top_margin - bottom_margin # Adjust height for margins

    # Create a SimpleDocTemplate with adjusted page size and margins
    doc = SimpleDocTemplate(
        str(output_path),
        pagesize=(page_width, page_height),
        leftMargin=left_margin,
        rightMargin=right_margin,
        topMargin=top_margin,
        bottomMargin=bottom_margin
    )
    
    elements = []
    header = "Company Name".ljust(35) + "StockCode".ljust(10) + "Close".rjust(10) + "PrevClose".rjust(10) + \
             "MktCap".rjust(10) + "EMA10".rjust(10) + "EMA20".rjust(10) + "EMA50".rjust(10) + "EMA200".rjust(10)
    
    # add title
    formatted_df = get_formatted_dataframe(stock_list=stock_list)

    styles = getSampleStyleSheet()
    h2_style = styles['Title']
    h2_style.fontSize = 10
    h2_style.Name = 'Calibri'
    h2_style.leading = 9
    h2_style.alignment = 1  # 0=Left, 1=Center, 2=Right
    main_title = Paragraph(title, h2_style)
    elements.extend([main_title])

    # add space
    elements.extend([Spacer(0.5, 24)])

    #add header
    styles = getSampleStyleSheet()
    h2_style = styles['Title']
    h2_style.fontSize = 10
    title = Paragraph(header, style=h2_style)
    table_width = doc.width - doc.leftMargin - doc.rightMargin

    # Add elements for title and table
    records_per_page = 40
    chunks = [formatted_df[j:j + records_per_page] for j in range(0, len(formatted_df), records_per_page)]
    for chunk in chunks:
        elements.extend([title, create_table_with_header_formatted(chunk, add_column_names=True, table_width=table_width), PageBreak()])
        

    doc.build(elements, onFirstPage=add_headers_and_footers, onLaterPages=add_headers_and_footers)

In [256]:
print(heading_3_percent)
print("")
print_formatted_rows(ticker_list,title_3_percent)
print()
print()
print()
list_process_new(ticker_list, heading_3_percent)
list_process_to_pdf(ticker_list, header=heading_3_percent, output_path="titcker_list.pdf")
output_path = f"{report_date}_{title_3_percent}.pdf"
print_formatted_rows_pdf(stock_list=ticker_list,title=title_3_percent, output_path=output_path)

          7-day deliveries up_3_percent or new_highs above 500 cr 231020



In [258]:
print_formatted_rows(dely_3x_list,heading_3x)
print()
print()
print()
list_process_new(dely_3x_list, heading_3x)
list_process_to_pdf(dely_3x_list, header=heading_3x, output_path=f"{report_date}_heading_3x.pdf")
print_formatted_rows_pdf(stock_list=dely_3x_list,title=heading_3x, output_path=f"{report_date}_fromatted_heading_3x.pdf")

          7-day deliveries 3x dely volume 231020


Company Name                       StockCode      Close PrevClose    MktCap     EMA10     EMA20     EMA50    EMA200


Plaza Wires Limited                PLAZACABLE    107.35    102.25       443      0.00      0.00      0.00      0.00
Supershakti Metaliks Ltd.          541701        465.00    475.00       530    469.52    469.63    442.84    367.48
Himachal Fibres Ltd.               514010         14.99     14.70       256     30.21     28.53     23.47     13.35
Kintech Renewables Ltd.            512329       7751.35   7599.40     3,531   8384.38   8258.26   7731.22   4664.63
GKW Ltd.                           GKWLIMITED   1682.20   1481.60       884      0.00      0.00      0.00      0.00

Gulf Oil Lubricants India Ltd.     GULFOILLUB    567.30    567.05     3,573      0.00      0.00      0.00      0.00
JSW Holdings Ltd.                  JSWHL        4771.35   4741.20     5,860      0.00      0.00      0.00      0.00
Celebrity Fashions

In [260]:

print(heading_non_fno)
print("")
print_formatted_rows(unique_non_fno,title_non_fno)
print()
print()
print()
list_process_new(unique_non_fno, heading_non_fno)
list_process_to_pdf(unique_non_fno, header=heading_non_fno, output_path="heading_non_fno.pdf")
print_formatted_rows_pdf(stock_list=dely_3x_list,title=title_non_fno, output_path=f"{report_date}_fromatted_title_non_fno.pdf")

          Top 50 7-day deliveries non_fno 231020

Non_fno top deliveries 231020 with averages


Company Name                       StockCode      Close PrevClose    MktCap     EMA10     EMA20     EMA50    EMA200


CMS Info Systems Ltd.              CMSINFO       378.50    376.75     5,952      0.00      0.00      0.00      0.00
Zomato Ltd.                        ZOMATO        113.35    111.65   108,929      0.00      0.00      0.00      0.00
                                   ADANITRANS    820.80    811.95         0      0.00      0.00      0.00      0.00
Suzlon Energy Ltd.                 SUZLON         32.50     31.60    49,718      0.00      0.00      0.00      0.00
K.P.R. Mill Ltd.                   KPRMILL       830.00    824.95    28,220      0.00      0.00      0.00      0.00

JSW Energy Ltd.                    JSWENERGY     387.80    401.85    68,278      0.00      0.00      0.00      0.00
Hindustan Construction Company     HCC            27.80     28.60     4,321      0.00    

In [279]:
print(heading_sme)
print("")
print_formatted_rows(sme_list,title_sme)
print()
print()
print()
list_process_new(sme_list,heading_sme)
list_process_to_pdf(sme_list, header=heading_sme, output_path="heading_sme.pdf")
print_formatted_rows_pdf(stock_list=dely_3x_list,title=title_sme, output_path=f"{report_date}_fromatted_title_sme.pdf")


          Top 50 7-day deliveries sme 231020

SME top deliveries 231020 with averages


Company Name                       StockCode      Close PrevClose    MktCap     EMA10     EMA20     EMA50    EMA200


Tridhya Tech Ltd.                  TRIDHYA        40.50     40.05        79     35.03     35.89     37.70     40.85
CFF Fluid Control Ltd.             543920        390.50    379.75       816    424.67    427.27    419.64    333.23
Jeena Sikho Lifecare Ltd.          JSLL         1207.45   1189.10     1,493    611.82    624.90    652.13    682.24
Shigan Quantum Technologies Lt     SHIGAN        108.80    116.00       212    111.44    108.36    108.77    104.15
Vasa Denticity Ltd.                DENTALKART    487.00    486.05     1,015      0.00      0.00      0.00      0.00

Veefin Solutions Ltd.              543931        217.80    224.10       538    257.34    254.04    238.22    178.43
Drone Destination Ltd.             DRONE         124.50    124.80       354    154.78    157.33  

In [269]:
print_formatted_rows(holding_list,holding_list)
print()
print()
print()
list_process_new(holding_list, holding_list)
list_process_to_pdf(holding_list, header=holding_list, output_path="holding_list.pdf")
print_formatted_rows_pdf(stock_list=holding_list,title=holding_list, output_path=f"{report_date}_fromatted_holding_list.pdf")

NameError: name 'holding_list' is not defined

In [263]:
print(heading_3_percent_below)
print("")
print_formatted_rows(ticker_below,title_3_percent_below)
print()
print()
print()
list_process_new(ticker_below, heading_3_percent_below)
list_process_to_pdf(ticker_below, header=heading_3_percent_below, output_path="heading_3_percent_below.pdf")
print_formatted_rows_pdf(stock_list=ticker_below,title=title_3_percent_below, output_path=f"{report_date}_fromatted_title_3_percent_below.pdf")

          7-day deliveries up_3_percent or new_highs below 500 cr 231020

         up_3_percent or new_highs below 500 cr 231020 with averages


Company Name                       StockCode      Close PrevClose    MktCap     EMA10     EMA20     EMA50    EMA200


Sikko Industries Ltd.              SIKKO          67.00     67.05       191      0.00      0.00      0.00      0.00
Starteck Finance Ltd.              STARTECK      160.05    160.00       409      0.00      0.00      0.00      0.00
Hindcon Chemicals Ltd.             HINDCON       133.70    133.00       314      0.00      0.00      0.00      0.00
Coral India Finance & Housing      CORALFINAC     43.25     44.35       254      0.00      0.00      0.00      0.00
MPS Infotecnics Ltd.               VISESHINFO      0.45      0.50       189      0.00      0.00      0.00      0.00

Orissa Bengal Carrier Ltd.         OBCL           61.55     56.65       128      0.00      0.00      0.00      0.00
Arihant Academy Ltd.               ARIHA

In [275]:
print_formatted_rows(list_above_10k,title_above_10k)
print()
print()
print()
list_process_new(list_above_10k, heading_above_10k)
list_process_to_pdf(list_above_10k, header=heading_above_10k, output_path="heading_above_10k.pdf")
print_formatted_rows_pdf(stock_list=list_above_10k,title=title_above_10k.strip(), output_path=f"{report_date}_list_above_10k.pdf")

         Top deliveries mktcap above 10k cr 231020 with averages


Company Name                       StockCode      Close PrevClose    MktCap     EMA10     EMA20     EMA50    EMA200


Adani Power Ltd.                   ADANIPOWER    338.30    344.45   197,823      0.00      0.00      0.00      0.00
Amber Enterprises India Ltd.       AMBER        3036.60   2951.30    10,245      0.00      0.00      0.00      0.00
Angel One Ltd.                     ANGELONE     2343.45   2200.45    27,611      0.00      0.00      0.00      0.00
Bikaji Foods International Ltd     BIKAJI        489.05    491.50    13,823      0.00      0.00      0.00      0.00
Blue Star Ltd.                     BLUESTARCO    919.30    944.25    19,465      0.00      0.00      0.00      0.00

Brigade Enterprises Ltd.           BRIGADE       611.70    614.60    19,992      0.00      0.00      0.00      0.00
BSE Ltd.                           BSE          1587.80   1537.60    31,280      0.00      0.00      0.00      0.00
Ce

In [276]:
print_formatted_rows(list_1k_10k, title_1k_10k)
print()
print()
print()

list_process_new(list_1k_10k, heading_1k_10k)
list_process_to_pdf(list_1k_10k, header=heading_1k_10k, output_path="heading_1k_10k.pdf")
print_formatted_rows_pdf(stock_list=list_1k_10k,title=title_1k_10k.strip(), output_path=f"{report_date}_title_1k_10k.pdf")

          Top deliveries mktcap 1k - 10k cr 231020 with averages


Company Name                       StockCode      Close PrevClose    MktCap     EMA10     EMA20     EMA50    EMA200


Balu Forge Industries Ltd.         531112        218.60    215.25     2,575    246.74    243.97    234.44    182.80
BMW Industries Ltd.                542669         39.57     39.49     1,231     54.90     54.94     51.43     41.25
Insolation Energy Ltd.             543620        531.70    484.00     1,448    684.06    672.14    607.58    377.44
Anant Raj Ltd.                     ANANTRAJ      237.45    235.80     9,713      0.00      0.00      0.00      0.00
The Anup Engineering Ltd.          ANUP         2123.20   2157.35     2,715      0.00      0.00      0.00      0.00

Apollo Micro Systems Ltd.          APOLLO         75.35     76.85     2,987      0.00      0.00      0.00      0.00
Arvind Ltd.                        ARVIND        188.90    190.45     6,952      0.00      0.00      0.00      0.00
Ar

In [277]:
print_formatted_rows(list_below_1000, title_below_1000)
print()
print()
print()
list_process_new(list_below_1000, heading_below_1000)
list_process_to_pdf(list_below_1000, header=heading_below_1000, output_path="heading_below_1000.pdf")
print_formatted_rows_pdf(stock_list=list_below_1000,title=title_below_1000.strip(), output_path=f"{report_date}_title_below_1000.pdf")

         Top deliveries mktcap below 1000 cr 231020 with averages


Company Name                       StockCode      Close PrevClose    MktCap     EMA10     EMA20     EMA50    EMA200


20 Microns Ltd.                    20MICRONS     134.95    137.70       667      0.00      0.00      0.00      0.00
Kilburn Engineering Ltd.           522101        205.80    197.00     1,067    282.14    278.92    255.32    179.65
Nile Ltd.                          530129        710.95    714.15       262    878.40    881.02    844.51    735.64
Ceinsys Tech Ltd.                  538734        261.30    268.10       542    356.46    356.08    336.10    251.65
RMC Switchgears Ltd.               540358        644.95    592.05       608    584.15    586.85    603.08    515.17

Khazanchi Jewellers Ltd            543953        229.00    228.50       678    274.50    276.12    264.36    207.34
Bondada Engineering Limited        543971        269.40    259.60       817    395.87    397.40    359.02    242.00
A

In [26]:
print("                         Top Deliveries with averages ", report_date)
print('________________________________________________________________________________________________________________')
print("")
print("")
print("")

print_formatted_rows(unique_non_fno,title_non_fno)
print('________________________________________________________________________________________________________________')
print('________________________________________________________________________________________________________________')
print()

print_formatted_rows(sme_list,title_sme)
print('________________________________________________________________________________________________________________')
print('________________________________________________________________________________________________________________')
print()

print_formatted_rows(list_above_10k,title_above_10k)
print('________________________________________________________________________________________________________________')
print('________________________________________________________________________________________________________________')
print()

print_formatted_rows(list_1k_10k,title_1k_10k)
print('________________________________________________________________________________________________________________')
print('________________________________________________________________________________________________________________')
print()

print_formatted_rows(list_below_1000, title_below_1000)
print('________________________________________________________________________________________________________________')
print('________________________________________________________________________________________________________________')
print()


                         Top Deliveries with averages  231226
________________________________________________________________________________________________________________



Non_fno top deliveries 231226 with averages


Company Name                       StockCode      Close PrevClose    MktCap     EMA10     EMA20     EMA50    EMA200


Indian Renewable Energy Dev Ag     IREDA         101.95    109.85    27,399      0.00      0.00      0.00      0.00
Mazagon Dock Shipbuilders Ltd.     MAZDOCK      2287.85   2121.40    46,195      0.00      0.00      0.00      0.00
Castrol India Ltd.                 CASTROLIND    182.55    162.55    18,017      0.00      0.00      0.00      0.00
Adani Green Energy Ltd.            ADANIGREEN   1600.20   1533.10   253,289      0.00      0.00      0.00      0.00
Indian Railway Finance Corpora     IRFC           97.40     97.75   127,287      0.00      0.00      0.00      0.00

Cochin Shipyard Ltd.               COCHINSHIP   1374.50   1301.40    18,061  

SJVN Ltd.                          SJVN           91.55     89.70    36,029      0.00      0.00      0.00      0.00
Network 18 Media & Investments     NETWORK18      88.85     86.10     9,285      0.00      0.00      0.00      0.00
IRB Infrastructure Developers      IRB            41.85     41.65    25,249      0.00      0.00      0.00      0.00
Orient Green Power Company Ltd     GREENPOWER     21.95     22.35     2,154      0.00      0.00      0.00      0.00

National Fertilizers Ltd.          NFL            85.90     84.25     4,213      0.00      0.00      0.00      0.00
Ultracab (India) Ltd.              538706         27.83     24.30       266     24.50     23.17     20.75     18.94
IFCI Ltd.                          IFCI           28.15     28.40     7,003      0.00      0.00      0.00      0.00
CESC Ltd.                          CESC          123.40    119.15    16,357      0.00      0.00      0.00      0.00
Trident Ltd.                       TRIDENT        36.05     36.15    18

Central Depository Services (I     CDSL         1851.05   1836.95    19,343      0.00      0.00      0.00      0.00
Cello World Limited                CELLO         765.50    769.20    16,257      0.00      0.00      0.00      0.00
Concord Biotech                    CONCORDBIO   1506.10   1504.70    15,715      0.00      0.00      0.00      0.00

CreditAccess Grameen Ltd.          CREDITACC    1613.25   1627.50    25,711      0.00      0.00      0.00      0.00
Elecon Engineering Company Ltd     ELECON        940.25    893.50    10,549      0.00      0.00      0.00      0.00
Vodafone Idea Ltd.                 IDEA           13.35     13.60    65,037      0.00      0.00      0.00      0.00
Jindal Saw Ltd.                    JINDALSAW     422.15    424.40    13,481      0.00      0.00      0.00      0.00
JK Cement Ltd.                     JKCEMENT     3765.40   3782.20    29,238      0.00      0.00      0.00      0.00

Jupiter Wagons Ltd.                JWL           320.30    323.70    1

Indraprastha Medical Corporati     INDRAMEDCO    178.65    179.55     1,645      0.00      0.00      0.00      0.00

Inox Wind Ltd.                     INOXWIND      466.15    453.25    15,183      0.00      0.00      0.00      0.00
JK Lakshmi Cement Ltd.             JKLAKSHMI     885.55    870.05    10,372      0.00      0.00      0.00      0.00
JK Tyre & Industries Ltd.          JKTYRE        377.40    383.30     9,290      0.00      0.00      0.00      0.00
Jaiprakash Associates Ltd.         JPASSOCIAT     20.85     21.10     5,118      0.00      0.00      0.00      0.00
Jaiprakash Power Ventures Ltd.     JPPOWER        13.95     13.25     9,519      0.00      0.00      0.00      0.00

Sai Kalamandir                     KALAMANDIR    274.90    277.50     4,219      0.00      0.00      0.00      0.00
Kesoram Industries Ltd.            KESORAMIND    168.95    167.05     5,241      0.00      0.00      0.00      0.00
KFin Technologies Ltd.             KFINTECH      502.60    491.85     

Pyramid Technoplast Limited        PYRAMID       195.85    192.90       722      0.00      0.00      0.00      0.00

Semac Consultants Ltd              SEMAC        2303.00   2350.00       708      0.00      0.00      0.00      0.00
Sky Gold Ltd.                      SKYGOLD      1003.90    956.10     1,079      0.00      0.00      0.00      0.00
Venus Remedies Ltd.                VENUSREM      395.80    389.95       527      0.00      0.00      0.00      0.00
Walchandnagar Industries Ltd.      WALCHANNAG    196.95    187.60       905      0.00      0.00      0.00      0.00
________________________________________________________________________________________________________________
________________________________________________________________________________________________________________



In [25]:
check_stock = ['inspirisys','coalindia','wockpharma', 'imagicaa','stltech','cub','sagcem','sapphire']
list_process_new(check_stock, "temp check")

temp check
Company Name: Inspirisys Solutions Ltd. (NSE Code: INSPIRISYS, BSE Code: 532774, Mktcap:338, Networth:-9)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231020           5,181           3,753           8,934      71.45
   231010           7,170           1,545           8,715      63.30
   230810           1,354           1,729           3,083      78.45
   230420           6,390           2,960           9,350      54.15
   190124             932               2             934      56.00



Company Name: Coal India Ltd. (NSE Code: COALINDIA, BSE Code: 533278, Mktcap:225,618, Networth:69,686)

ema10 / ema20 / ema50 / ema200     0  0  0  0

     Date    NSE_Delivery    BSE_Delivery  Total_Delivery      Price
   231020       2,305,726          46,430       2,352,156     312.40
   231010       8,636,371         172,611       8,808,982     303.25
   230810       5,663,735          71,336       5,735,071   

In [85]:
from reportlab.lib.pagesizes import letter, A4
from reportlab.pdfgen import canvas
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle

def create_table_with_header(df, add_column_names=False, table_width=None):
    """create a table with header and data"""
    table_data = [df.columns.tolist()] + df.values.tolist()
        
    style = [
        ('ALIGN', (0, 0), (0, -1), 'LEFT'),  # Left align first column
        ('ALIGN', (1, 0), (-1, -1), 'RIGHT'),  # Right align remaining columns
    ]

    t = Table(table_data, repeatRows=1, colWidths=[100, 80, 80, 80, 60])
    column_header_style = [('BACKGROUND', (0, 0), (-1, 0), colors.lightgrey),
                        ('TEXTCOLOR', (0, 0), (-1, 0), colors.black),
                        ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
                        ]
    
    t.setStyle(column_header_style)
    t.setStyle(style)
    
    # Remove table borders
    t.setStyle([('LINEBELOW', (0, 0), (-1, 0), 0, colors.white),  # Hide line below header
                ('GRID', (0, 0), (-1, -1), 0, colors.white)])      # Hide grid lines
    
    return t

def add_headers_and_footers(canvas, doc):
    """Add header and footer here"""
    page_num = canvas.getPageNumber()
    text = f"Page {page_num}"
    footer_style = getSampleStyleSheet()['Normal']
    footer_style.alignment = 1  # 0=Left, 1=Center, 2=Right

    footer = Paragraph(text, footer_style)
    w, h = footer.wrap(doc.width, doc.bottomMargin)
    footer.drawOn(canvas, doc.leftMargin, doc.bottomMargin - h)

    # Change title text color to white for pages after the first one
    if page_num > 1:
        title_style = getSampleStyleSheet()['Title']
        title_style.textColor = colors.white

def list_process_to_pdf(list_name, header, output_path):
    left_margin = 20
    right_margin = 20
    top_margin = 5
    bottom_margin = 20  # Add a bottom margin
    page_width = A4[0]  # Get width from letter page size
    page_height = A4[1] - top_margin - bottom_margin  # Adjust height for margins

    # Create a SimpleDocTemplate with adjusted page size and margins
    doc = SimpleDocTemplate(
        str(output_path),
        pagesize=(page_width, page_height),
        leftMargin=left_margin,
        rightMargin=right_margin,
        topMargin=top_margin,
        bottomMargin=bottom_margin
    )

    # Create a list to hold all elements
    elements = []
    for company in list_name:
        if company.isdigit():
            company_int = int(company)
            company_data = combined_df[combined_df['BSE_Code'] == company_int]
        else:
            company = company.upper()
            company_data = combined_df[combined_df['NSE_Code'] == company]

        if not company_data.empty:
            company_name = company_data['Company Name'].values[0]
            nse_code = company_data['NSE_Code'].values[0]
            bse_code = company_data['BSE_Code'].values[0]

            ema10, ema20, ema50, ema200 = averages(nse_code)
            ema_str = "ema10 / ema20 / ema50 / ema200"

            mktcap_value = get_mktcap(nse_code)

            # adding company info to pdf
            txt = f"Company Name: {company_name} (NSE Code: {nse_code}, BSE Code: {bse_code}, Mktcap: {mktcap_value[0]:,.0f})"
            styles = getSampleStyleSheet()
            h2_style = styles['Normal']
            h2_style.fontSize = 11
            h2_style.Name = 'Arial'
            h2_style.bold = True
            h2_style.alignment = 0 
            main_title = Paragraph(txt, h2_style)
            elements.extend([main_title])
            txt =f"{ema_str}     {ema10[0]}  {ema20[0]}  {ema50[0]}  {ema200[0]}"

            h2_style.fontSize = 10
            main_title = Paragraph(txt, h2_style)
            elements.extend([main_title])

            formatted_columns = ['Date', 'NSE_Delivery', 'BSE_Delivery', 'Total_Delivery', 'Price']
            formatted_data = company_data[formatted_columns].copy()
            formatted_data['NSE_Delivery'] = formatted_data['NSE_Delivery'].apply(lambda x: f"{int(x):,}")
            formatted_data['BSE_Delivery'] = formatted_data['BSE_Delivery'].apply(lambda x: f"{int(x):,}")
            formatted_data['Total_Delivery'] = formatted_data['Total_Delivery'].apply(lambda x: f"{int(x):,}")
            formatted_data['Price'] = formatted_data['Price'].apply(lambda x: f"{x:.2f}")

            formatted_data_str = formatted_data.to_string(index=False, col_space={'Date': 9, 'NSE_Delivery': 15, 'BSE_Delivery': 15,
                                                                                   'Total_Delivery': 15, 'Price': 10})
            table_width = doc.width - doc.leftMargin - doc.rightMargin
            table = create_table_with_header(formatted_data, add_column_names=True, table_width=table_width)
            elements.append(table)
            # adding space
            elements.extend([Spacer(1, 24)])
        else:
            txt=f"No data found for {company}\n"
            main_title = Paragraph(txt, h2_style)
            elements.extend([main_title])
        
    doc.build(elements, onFirstPage=add_headers_and_footers, onLaterPages=add_headers_and_footers)        

In [86]:
list_process_to_pdf(check_stock, "temp check", "output_a4_spacing_1_reduced_testing.pdf")    